<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/main/17_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install eciespy eth_keys cryptos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 2.9 MB/s 
     |████████████████████████████████| 555 kB 15.3 MB/s 
     |████████████████████████████████| 2.0 MB 31.3 MB/s 
     |████████████████████████████████| 105 kB 62.7 MB/s 
     |████████████████████████████████| 481 kB 59.8 MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.11.2-cp37-cp37m-linux_x86_64.whl size=1236692 sha256=d801ae332e544395534917224a8e36aa1cffb8e759311543fe273e0c9a38e666
  Stored in directory: /root/.cache/pip/wheels/38/70/71/ca13ea3d36ccd0b3d0ec7d7a4ca67522048d695b556bba4f59
Successfully built cytoolz


In [2]:
import ecies
import eth_keys
import coincurve
import hashlib

In [19]:
broad_message=b'''Tell the world something. Encrypt with AES key.
AES encrypt multiple copes of that key using a list of shared secrets.'''
print(broad_message.decode(),len(broad_message))

Tell the world something. Encrypt with AES key.
AES encrypt multiple copes of that key using a list of shared secrets. 118


In [39]:
from coincurve.utils import get_valid_secret

master_aes=get_valid_secret()
broad_message_aes=ecies.aes_encrypt(master_aes,broad_message)

Now we have an encrypted message and we want to distribute it so select key holders can open it. 

We will encrypt the `master_aes` encryption key (using aes) so that each recipient can access it using their special-unique encrypted copy.

**HOT DIGGITY DOG !!! HOW DO WE MAKE SURE ONLY THE SELECT KEY HOLDER CAN OPEN THEIR COPY ???**

#Diffie Helman Key Exchange...Broadcast

In [20]:
privKey1 = ecies.utils.generate_eth_key()
privKey2 = ecies.utils.generate_eth_key()
privKey3 = ecies.utils.generate_eth_key()
print(privKey1, type(privKey1))
print(privKey2, type(privKey2))
print(privKey3, type(privKey3))

0x56b61df9c372f3a4c59fc986c7c1745c6b733de1dd8cdb8ce0dfaab39a48b86c <class 'eth_keys.datatypes.PrivateKey'>
0x6655df8927cd5df81737292294210336eb80298d379d5a17b105f60df0eb5751 <class 'eth_keys.datatypes.PrivateKey'>
0xa4d3efc650a78d101a9581c24dd273c95511783c58d21b8cc42bdbbbac060623 <class 'eth_keys.datatypes.PrivateKey'>


In [41]:
#convert from ecies private/public key to concurve private/public key

k1 = coincurve.PrivateKey(privKey1.to_bytes())
k2 = coincurve.PrivateKey(privKey2.to_bytes())
k3 = coincurve.PrivateKey(privKey2.to_bytes())

p1 = coincurve.PublicKey( privKey1.public_key.to_compressed_bytes() )
p2 = coincurve.PublicKey( privKey2.public_key.to_compressed_bytes() )
p3 = coincurve.PublicKey( privKey3.public_key.to_compressed_bytes() )

In [42]:
shared12 = p2.multiply(k1.secret)
shared21 = p1.multiply(k2.secret)

In [23]:
shared12.format().hex(),shared21.format().hex()

('02097d7834368fbdc1c1e016a5301a9f71be7ed7578fa0aa63ff43e1755b0f34b8',
 '02097d7834368fbdc1c1e016a5301a9f71be7ed7578fa0aa63ff43e1755b0f34b8')

In [32]:
from Crypto.Protocol.KDF import HKDF
AES_KEY_BYTES_LEN = 32
from Crypto.Hash import SHA256

In [44]:
ps=[p1,p2,p3]
shared_secrets=[p.multiply(k1.secret) for p in ps]
shared_keys=[ HKDF(shared_secret.format(), AES_KEY_BYTES_LEN, b"", SHA256) for shared_secret in shared_secrets]


In [47]:
access_keys=[ ecies.aes_encrypt(shared_key,master_aes) for shared_key in shared_keys]
print(len(b''.join(access_keys)))
access_keys

192


[b'`\xcc"\xcc\xd6]\x98M^\xc5\x88\x01\x0b\xd49G\xedG\tK\xa8\xfc(\x1dC\xces1,wr\xd8!\xe1.\xae\x1c\xae#7\xd6\xb5(\xa4R\x9f\xfe\xce\x89-d\x7f\xb0\xfb\xae?s\xe4X\xd4\xae\xc3\xc8\x85',
 b'\x8a\xe1\xe4\x91\x1e"kP"\x82e\xb9\xde\xcd\xc8A\xcaq\x8a`\xc2\xa5\x87\x00\xb65\xcb\xce\x8b\xbc\x1e\x15\xc2\xdei\x81\x89qn\xbb0.\xfd\xff\x88\x86A\x86\xc3L,\xf6\x83\xfe\xca\x13A\xf7\x08\x13\xd2oc\xbb',
 b'\x9bTZ\x08\x8f\xa5\xc1\xccY\x0c.h\xd7\xe5\x85\x8f\x92\x13\xa4\x9c\xae\x8a\x17\x1dY\xb6\xa5\xcf\x17\x04\x15\xd0\x0f\n_\xc5\xb5b2jt&5\x8b\x12\x98\xf2\x08NE\x8c7\xab\x9cTT\xd9^\xac\xa5\x94\xad\xc7*']

In [58]:
ecies.aes_decrypt(,access_keys[0])

ValueError: ignored

In [56]:
dist_keys=[ecies.aes_decrypt(*args)  for args in zip([privKey1.to_bytes(),
                                                      privKey2.to_bytes(),
                                                      privKey3.to_bytes()],access_keys)]

ValueError: ignored

In [26]:
shared_aes_key=hashlib.sha256(shared12.format()).digest()

In [28]:
len(shared_aes_key),len(shared12.format())

(32, 33)

In [29]:
len(ecies.aes_encrypt(shared_aes_key,shared_aes_key))

64

In [36]:
HKDF(shared21.format(), AES_KEY_BYTES_LEN, b"", SHA256)

b'\x19Vu\x98\xeb\x14\xb3\x1d\xde7\xef\x93\xdf\reX\xa0\xac\xa6P\xe3(\xd8\xf4\x9cw\x93\x03\xb9\x92\xbc?'

In [37]:
HKDF(shared12.format(), AES_KEY_BYTES_LEN, b"", SHA256)

b'\x19Vu\x98\xeb\x14\xb3\x1d\xde7\xef\x93\xdf\reX\xa0\xac\xa6P\xe3(\xd8\xf4\x9cw\x93\x03\xb9\x92\xbc?'